In [3]:
from spotipy import (
    Spotify,
    SpotifyClientCredentials
)
from _spotify.spotipy_auth import spotipy_auth


spotipy_auth()
sp = Spotify(auth_manager = SpotifyClientCredentials())


# get audio features from albums
def _reduce_albums_to_id_and_name(item: dict) -> dict:
    return {
        'id': item['id'], 
        'name': item['name']
    }


def artist_album(artist_id: str) -> list:
    albums = []
    res = sp.artist_albums(artist_id = artist_id)
    for i in res['items']:
        albums.append(_reduce_albums_to_id_and_name(i))

    while res['next']:
        res = sp.next(res)
        for i in res['items']:
            albums.append(_reduce_albums_to_id_and_name(i))

    return albums

In [85]:
d

In [48]:
artist_id = "4Z8W4fKeB5YxbusRsdQVPb"
albums = artist_album(artist_id = artist_id)

In [89]:
[album_tracks(album['id']) for album in albums]

[[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
     'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
     'id': '4Z8W4fKeB5YxbusRsdQVPb',
     'name': 'Radiohead',
     'type': 'artist',
     'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CA',
    'CD',
    'CG',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
    'CR',
    'CV',
    'CW',
    'CY',
    'CZ',
    'DE',
    'DJ',
    'DK',
    'DM',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FJ',
    'FM',
    'FR',
    'GA',
    'GB',
    'GD',
    'GE',
    'GH',
    'GM',
    'GN',
    'GQ',
    'GR',
    'GT',
    'GW',
    'GY',
